<a href="https://colab.research.google.com/github/Tommy-Las/WatfordFC/blob/main/Data_cleaning_transformation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# If google colab

In [131]:
# Mount drive

# from google.colab import drive
# drive.mount('/content/drive')

# Import packages and data

In [132]:
import pandas as pd
# Turn off warnings
pd.options.mode.chained_assignment = None
pd.set_option('display.max_columns', None)
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.impute import KNNImputer
from sklearn.preprocessing import OneHotEncoder

Import GPS data

In [133]:
# file_path = '/content/drive/MyDrive/WATFORD FC/Datos GPS/GPS 2018-2023.xlsx'
file_path = 'data/GPS 2018-2023_NoContact.xlsx'
df_gps = pd.read_excel(file_path)

Import speed data

In [134]:
# file_path = '/content/drive/MyDrive/WATFORD FC/Datos GPS/max_speed.xlsx'
file_path = 'data/max_speed.xlsx'
df_speed = pd.read_excel(file_path)

In [135]:

file_path = 'data/processed_newdata.xlsx'
df_new = pd.read_excel(file_path)
df_new.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3251 entries, 0 to 3250
Data columns (total 14 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   PlayerID     3251 non-null   int64         
 1   Injury       3251 non-null   int64         
 2   Session      3218 non-null   object        
 3   Date         3251 non-null   datetime64[ns]
 4   Mins         3251 non-null   float64       
 5   TD           3251 non-null   float64       
 6   >19.8        3251 non-null   float64       
 7   >25          3251 non-null   float64       
 8   >19.8_Rel-1  3076 non-null   float64       
 9   >25_Rel-1    3076 non-null   float64       
 10  Sprints      3251 non-null   float64       
 11  % Max Speed  3240 non-null   float64       
 12  ACC          3251 non-null   float64       
 13  DEC          3251 non-null   float64       
dtypes: datetime64[ns](1), float64(10), int64(2), object(1)
memory usage: 355.7+ KB


Import wellbeing data


In [136]:
#file_path = '/content/drive/MyDrive/WATFORD FC/Datos Wellbeing/wellbeing ssp.xlsx'
#df_wellbeing = pd.read_excel(file_path)

# Merge Sprint rows into a single row per session

The sprint values are separated in different rows per session ,

We want to merge in into a single row per session.

## Transform data before merging sprint rows

In [137]:
df_speed.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 123180 entries, 0 to 123179
Data columns (total 11 columns):
 #   Column                                       Non-Null Count   Dtype  
---  ------                                       --------------   -----  
 0   DATE                                         123180 non-null  object 
 1   ID                                           123180 non-null  int64  
 2   Player Position                              123155 non-null  object 
 3   Max Speed                                    116475 non-null  float64
 4   Sprints                                      113560 non-null  float64
 5   MINUTES                                      113654 non-null  float64
 6   Season                                       123180 non-null  object 
 7   Max Speed Season                             123180 non-null  float64
 8   Avg Speed Season                             122357 non-null  float64
 9   % Max Speed                                  122357 non-nul

We want to remove rows after 08-07-2021 since we don't have data for sprints and speed

In [138]:
from datetime import date
# Convert DATE column values into Pandas datetime object
df_speed['DATE'] = pd.to_datetime(df_speed['DATE'], dayfirst=True)

# Filter rows after date: '2021-08-07'
date_filter = pd.Timestamp('2021-08-07')
df_speed = df_speed[df_speed['DATE'] > date_filter]

# Filter rows after date: '2023-05-11'
date_filter = pd.Timestamp('2023-05-11')
df_speed = df_speed[df_speed['DATE'] < date_filter]

date_filter = pd.Timestamp('2021-06-28')
df_gps = df_gps[df_gps['DATE'] > date_filter]

In [139]:
df_speed.info()

<class 'pandas.core.frame.DataFrame'>
Index: 55390 entries, 58257 to 113646
Data columns (total 11 columns):
 #   Column                                       Non-Null Count  Dtype         
---  ------                                       --------------  -----         
 0   DATE                                         55390 non-null  datetime64[ns]
 1   ID                                           55390 non-null  int64         
 2   Player Position                              55365 non-null  object        
 3   Max Speed                                    55377 non-null  float64       
 4   Sprints                                      55362 non-null  float64       
 5   MINUTES                                      55389 non-null  float64       
 6   Season                                       55390 non-null  object        
 7   Max Speed Season                             55390 non-null  float64       
 8   Avg Speed Season                             55390 non-null  float64       


Replace NULL sprint values to 0

In [140]:
# Replace NULL values from Sprint to 0
df_speed['Sprints'] = df_speed['Sprints'].fillna(0)

## Merge sprint values

Print values to compare merge later

In [141]:
df_speed.sort_values(by='DATE', ascending=False).head(15)

,DATE,ID,Player Position,Max Speed,Sprints,MINUTES,Season,Max Speed Season,Avg Speed Season,% Max Speed,%Speed diference against max. Speed average
58257,2023-05-10,23085,CENTRE MIDFIELDER,22.91,3.0,30.19,2022-2023,32.12,20.304388,71.326276,12.832753
58258,2023-05-10,87583,CENTRE MIDFIELDER,24.70,20.0,40.53,2022-2023,34.27,21.189939,72.074701,16.564751
58259,2023-05-09,96811,CENTRE MIDFIELDER,32.19,10.0,70.51,2022-2023,36.04,20.882454,89.317425,54.148548
58260,2023-05-09,89871,STRIKER,25.46,0.0,79.87,2022-2023,34.12,21.753402,74.618992,17.039162
58300,2023-05-08,55555,FULL BACK,30.47,18.0,72.62,2022-2023,34.94,22.071250,87.206640,38.052897
58289,2023-05-08,48692,CENTRE BACK,32.56,35.0,69.15,2022-2023,36.21,21.029270,89.919912,54.831811
58290,2023-05-08,87583,CENTRE MIDFIELDER,22.69,11.0,34.13,2022-2023,34.27,21.189939,66.209513,7.079117
58291,2023-05-08,37537,CENTRE MIDFIELDER,28.65,5.0,47.05,2022-2023,34.09,21.974456,84.042241,30.378655
58292,2023-05-08,37537,CENTRE MIDFIELDER,30.59,3.0,22.10,2022-2023,34.09,21.974456,89.733060,39.207088
58293,2023-05-08,37537,CENTRE MIDFIELDER,30.59,8.0,69.15,2022-2023,34.09,21.974456,89.733060,39.207088


Do a group by player and date, and select the maximum value for each feature

In [142]:
df_speed = df_speed.groupby(['DATE', 'ID']).agg('max').reset_index()

In [143]:
df_speed.sort_values(by='DATE', ascending=False).head(15)

,DATE,ID,Player Position,Max Speed,Sprints,MINUTES,Season,Max Speed Season,Avg Speed Season,% Max Speed,%Speed diference against max. Speed average
8262,2023-05-10,87583,CENTRE MIDFIELDER,24.70,20.0,40.53,2022-2023,34.27,21.189939,72.074701,16.564751
8261,2023-05-10,23085,CENTRE MIDFIELDER,22.91,3.0,30.19,2022-2023,32.12,20.304388,71.326276,12.832753
8260,2023-05-09,96811,CENTRE MIDFIELDER,32.19,10.0,70.51,2022-2023,36.04,20.882454,89.317425,54.148548
8259,2023-05-09,89871,STRIKER,25.46,0.0,79.87,2022-2023,34.12,21.753402,74.618992,17.039162
8249,2023-05-08,55555,FULL BACK,30.47,18.0,72.62,2022-2023,34.94,22.071250,87.206640,38.052897
8240,2023-05-08,10103,FULL BACK,21.24,1.0,44.84,2022-2023,34.95,22.202230,60.772532,-4.333932
8241,2023-05-08,12086,WINGER,35.04,41.0,88.27,2022-2023,35.44,21.901184,98.871332,59.991349
8242,2023-05-08,19817,FULL BACK,31.68,0.0,128.04,2022-2023,35.77,21.605460,88.565837,46.629599
8243,2023-05-08,21079,WINGER,34.51,51.0,88.04,2022-2023,36.97,22.708654,93.345956,51.968499
8244,2023-05-08,23085,CENTRE MIDFIELDER,23.71,7.0,58.52,2022-2023,32.12,20.304388,73.816936,16.772788


# Handle duplicates for GPS Data

There are some players that have 2 records for the same day, they are different amounts so we sum them

In [144]:
# Define columns to sum
columns_to_sum = ['Total D', '>19.8', '> 25 Km/h', 'ACC', 'DEC']

# Define columns to select the first value
columns_to_first = ['DATE', 'Column2', 'PLAYER', 'Injury', 'season', 'LEAGUE', 'preseason-season', 'MANAGER']

# Group by the duplicate subset and aggregate
df_gps_aggregated = (
    df_gps[df_gps.duplicated(subset=['PLAYER', 'DATE'], keep=False)]
    .groupby(['PLAYER', 'DATE'], as_index=False)
    .agg({**{col: 'sum' for col in columns_to_sum},
          **{col: 'first' for col in columns_to_first}})
)

# Ensure non-duplicated rows are preserved by combining them back
df_gps_combined = pd.concat([
    df_gps[~df_gps.duplicated(subset=['PLAYER', 'DATE'], keep=False)],
    df_gps_aggregated
], ignore_index=True)

# Merge GPS and Speed dataframes

## Prepare the data before the merge

Verify column names for both Dataframes

In [145]:
df_gps_combined.columns

Index(['DATE', 'Column2', 'PLAYER', 'Injury', 'season', 'LEAGUE',
       'preseason-season', 'MANAGER', 'Total D', '>19.8', '> 25 Km/h', 'ACC',
       'DEC'],
      dtype='object')

In [146]:
df_speed.columns

Index(['DATE', 'ID', 'Player Position', 'Max Speed', 'Sprints', 'MINUTES',
       'Season', 'Max Speed Season', 'Avg Speed Season', '% Max Speed',
       '%Speed diference against max. Speed average'],
      dtype='object')

Change datatypes before the merge

In [147]:
# Convert DATE column from GPS df into datetime pandas obkect
df_gps_combined['DATE'] = pd.to_datetime(df_gps_combined['DATE'], dayfirst=True)

# Drop NULL values for 'PLAYER'
df_gps_combined = df_gps_combined.dropna(subset=['PLAYER'])

# Convert ID and PLAYER columns to the same data type - integers
df_gps_combined['PLAYER'] = df_gps_combined['PLAYER'].astype(int)
df_speed['ID'] = df_speed['ID'].astype(int)

## Do the merge of gps & speed dataframes

In [148]:
# Perform an inner join on matching DATE and PLAYER/ID values
df_merged = df_gps_combined.merge(df_speed, left_on=['DATE', 'PLAYER'], right_on=['DATE', 'ID'], how='inner')

In [149]:
df_merged.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7831 entries, 0 to 7830
Data columns (total 23 columns):
 #   Column                                       Non-Null Count  Dtype         
---  ------                                       --------------  -----         
 0   DATE                                         7831 non-null   datetime64[ns]
 1   Column2                                      7831 non-null   object        
 2   PLAYER                                       7831 non-null   int64         
 3   Injury                                       7831 non-null   float64       
 4   season                                       7831 non-null   object        
 5   LEAGUE                                       7831 non-null   object        
 6   preseason-season                             7831 non-null   object        
 7   MANAGER                                      7831 non-null   object        
 8   Total D                                      7831 non-null   object        
 9

Verify class imbalances

In [150]:
print("No Injury: " + str(df_merged[df_merged["Injury"] == 0].shape[0]))
print("Injury: " + str(df_merged[df_merged["Injury"] == 1].shape[0]))
print(f"%: {((df_merged[df_merged['Injury'] == 1].shape[0] / df_merged.shape[0]) * 100):.2f}%")

No Injury: 7794
Injury: 37
%: 0.47%


# Merge df with 'weight' in wellbeing dataframe

In [151]:
# df_wellbeing = df_wellbeing[["Weight", "Date", "PLAYER"]]
# df_wellbeing.info()

In [152]:
# # Sort by PLAYER and DATE to ensure correct order for backfilling
# df_wellbeing = df_wellbeing.sort_values(by=['PLAYER', 'Date'])

# # Fill the NULL values in 'Weight' with the previous valid value for each player
# df_wellbeing['Weight'] = df_wellbeing.groupby('PLAYER')['Weight'].bfill()
# df_wellbeing['Weight'] = df_wellbeing.groupby('PLAYER')['Weight'].ffill()

# df_wellbeing.info()

In [153]:
# # Merge the DataFrames
# df_merged = df_merged.merge(df_wellbeing, left_on=['DATE', 'PLAYER'], right_on=['Date', 'PLAYER'], how='left')

# # Sort by PLAYER and DATE to ensure chronological order
# df_merged = df_merged.sort_values(by=['PLAYER', 'DATE'])

# # Back-fill null values in 'Weight' for each PLAYER
# df_merged['Weight'] = df_merged.groupby('PLAYER')['Weight'].bfill()
# df_merged['Weight'] = df_merged.groupby('PLAYER')['Weight'].ffill()

# # Optional: Drop the 'Date' column from the right DataFrame if not needed
# df_merged = df_merged.drop(columns=['Date'])

# # Display the final DataFrame
# df_merged.info()

In [154]:
#df_merged.groupby('PLAYER')['Weight'].apply(lambda x: x.isnull().sum())

# Transform numeric columns to the same data types

In [155]:
cols = ['Total D', '>19.8', '> 25 Km/h', 'ACC',
       'DEC', 'ID', 'Max Speed', 'Sprints', 'MINUTES', 'Max Speed Season',
       'Avg Speed Season', '% Max Speed',
       '%Speed diference against max. Speed average']

df_merged[cols] = df_merged[cols].astype(float)

In [156]:
df_merged.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7831 entries, 0 to 7830
Data columns (total 23 columns):
 #   Column                                       Non-Null Count  Dtype         
---  ------                                       --------------  -----         
 0   DATE                                         7831 non-null   datetime64[ns]
 1   Column2                                      7831 non-null   object        
 2   PLAYER                                       7831 non-null   int64         
 3   Injury                                       7831 non-null   float64       
 4   season                                       7831 non-null   object        
 5   LEAGUE                                       7831 non-null   object        
 6   preseason-season                             7831 non-null   object        
 7   MANAGER                                      7831 non-null   object        
 8   Total D                                      7831 non-null   float64       
 9

# Values that are 0 for ACC, DEC, Max Speed

Perform imputations on missing data

In [157]:
# Count of rows containing 0 for each column
zero_counts = (df_merged == 0).sum()

# Creating a DataFrame for the results
result_df = pd.DataFrame({
    "Column Name": zero_counts.index,
    "Number of Rows with 0": zero_counts.values
})

result_df

,Column Name,Number of Rows with 0
0,DATE,0
1,Column2,382
2,PLAYER,0
3,Injury,7794
4,season,0
5,LEAGUE,0
6,preseason-season,0
7,MANAGER,0
8,Total D,3
9,>19.8,420


Perform KNN Imputation

In [158]:
# Columns to impute
metrics_imputation_zeroes = ['Total D', 'ACC', 'DEC', 'Max Speed', 'MINUTES']

# Columns to use for KNN calculation
columns_for_knn = ['Total D', '>19.8', '> 25 Km/h', 'ACC', 'DEC', 'ID',
                   'Max Speed', 'Sprints', 'MINUTES', '% Max Speed']

# Replace 0 with NaN in the columns to be imputed
df_merged[metrics_imputation_zeroes] = df_merged[metrics_imputation_zeroes].replace(0, np.nan)

# Ensure the columns used for KNN calculation are numeric
columns_for_knn_numeric = df_merged[columns_for_knn].select_dtypes(include=[np.number]).columns.tolist()

# Initialize the KNNImputer
knn_imputer = KNNImputer(n_neighbors=5)  # Adjust n_neighbors if needed

# Perform KNN imputation using the broader set of columns for calculation
imputed_values = knn_imputer.fit_transform(df_merged[columns_for_knn_numeric])

# Update only the specified columns to impute
df_merged[metrics_imputation_zeroes] = imputed_values[:,
    [columns_for_knn_numeric.index(col) for col in metrics_imputation_zeroes]]

In [159]:
# Count of rows containing 0 for each column
zero_counts = (df_merged == 0).sum()

# Creating a DataFrame for the results
result_df = pd.DataFrame({
    "Column Name": zero_counts.index,
    "Number of Rows with 0": zero_counts.values
})

result_df

,Column Name,Number of Rows with 0
0,DATE,0
1,Column2,382
2,PLAYER,0
3,Injury,7794
4,season,0
5,LEAGUE,0
6,preseason-season,0
7,MANAGER,0
8,Total D,0
9,>19.8,420


Calculate '% Max Speed' and '% Speed difference against max. Speed average'

In [160]:
# Calculate '% Max Speed' for all rows
df_merged['% Max Speed'] = (df_merged['Max Speed'] / df_merged['Max Speed Season']) * 100

# Calculate '% Speed difference against max. Speed average' for all rows
df_merged['%Speed diference against max. Speed average'] = (
    (df_merged['Max Speed'] - df_merged['Avg Speed Season']) / df_merged['Avg Speed Season']
) * 100

# Reduce data volume/size

## Players who have never been injured

We are going to filter by players that had at least one injury, to reduce # of rows and hopefully improve accuracy

In [161]:
# Group by ID and sum the Injury column
injury_counts = df_merged.groupby('ID').agg({'Injury': 'sum'})

# Get the IDs where the sum of Injury is greater than 0
ids_to_remove = injury_counts[injury_counts['Injury'] == 0].index.tolist()

# Remove these IDs from the original DataFrame
df_filtered = df_merged[~df_merged['ID'].isin(ids_to_remove)]

In [162]:
print("No Injury: " + str(df_filtered[df_filtered["Injury"] == 0].shape[0]))
print("Injury: " + str(df_filtered[df_filtered["Injury"] == 1].shape[0]))
print(f"%: {((df_filtered[df_filtered['Injury'] == 1].shape[0] / df_filtered.shape[0]) * 100):.2f}%")

No Injury: 4683
Injury: 37
%: 0.78%


Verify that at least one player had an injury

In [163]:
df_filtered.groupby('ID').agg({'Injury': 'sum'})

,Injury
ID,
10103.0,4.0
10452.0,2.0
12086.0,1.0
17316.0,2.0
18096.0,3.0
20083.0,1.0
21079.0,2.0
23081.0,2.0
23085.0,2.0


# Change column names & drop unnecesary columns

In [164]:
column_rename_dict = {
    'Column2': 'Session',
    'DATE': 'DATE',
    'ID': 'PlayerID',
    'Total D': 'TD',
    '>19.8': '>19.8',
    '> 25 Km/h': '>25',
    'ACC': 'ACC',
    'DEC': 'DEC',
    'Max Speed': 'Max Speed',
    'Max Speed Season': 'Max Speed Season',
    'Avg Speed Season': 'Avg Speed Season',
    '% Max Speed': '% Max Speed',
    '%Speed diference against max. Speed average': 'Speed Diff Max Avg',
    'Injury': 'Injury',
    'MINUTES': 'Mins',
    'Sprints': 'Sprints',
    'Total D_Rel': 'TD_Rel',
    '>19.8_Rel': '>19.8_Rel',
    '> 25 Km/h_Rel': '>25 Km/h_Rel',
    'ACC_Rel': 'ACC_Rel',
    'DEC_Rel': 'DEC_Rel',
    'Sprints_Rel': 'Sprints_Rel'
}

df_filtered.rename(columns=column_rename_dict, inplace=True)

df_filtered.info()

<class 'pandas.core.frame.DataFrame'>
Index: 4720 entries, 3 to 7798
Data columns (total 23 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   DATE                4720 non-null   datetime64[ns]
 1   Session             4720 non-null   object        
 2   PLAYER              4720 non-null   int64         
 3   Injury              4720 non-null   float64       
 4   season              4720 non-null   object        
 5   LEAGUE              4720 non-null   object        
 6   preseason-season    4720 non-null   object        
 7   MANAGER             4720 non-null   object        
 8   TD                  4720 non-null   float64       
 9   >19.8               4720 non-null   float64       
 10  >25                 4720 non-null   float64       
 11  ACC                 4720 non-null   float64       
 12  DEC                 4720 non-null   float64       
 13  PlayerID            4720 non-null   float64       
 1

In [165]:
df_filtered = df_filtered.drop(columns=['LEAGUE', 'MANAGER', 'PLAYER', 'Season'])

# Calculate relative values

Remove spaces in 'Microcycle' column

In [166]:
df_filtered['Session'] = df_filtered['Session'].str.replace(' ', '', regex=False)
df_filtered["Session"].unique()

array(['MD', 'M-1', 'M-2', 'M-3', 'M-5', 'M+3', 'M+2', 'M-4', 'M+1',
       'MD(AWAY)', 'MD(HOME)', nan], dtype=object)

To calculate the relative values, we want the max and avg values for players that played over 85 in a Match Day

In [167]:
def calculate_relative_values(df_original, metrics):
    # Array that holds match days
    session_values = ['MD', 'MD(HOME)', 'MD(AWAY)']

    # Do a copy of the original DF
    df_copy = df_original.copy()

    # Filter players with +85 mins and match day
    df_filtered_85_md = df_copy[
        (df_copy['Mins'] > 85) &
        (df_copy['Session'].isin(session_values))
    ]

    # Players who never completed 85 mins in matchday
    df_filtered_85_only_mins = df_copy[df_copy['Mins'] > 85]

    # Combine both DataFrames
    df_filtered_85 = pd.concat([df_filtered_85_md, df_filtered_85_only_mins])

    # Temporary dictionaries to hold max and avg values per player
    player_max = {}
    player_avg = {}

    # Calculate max and avg for each player
    for player in df_filtered_85['PlayerID'].unique():
        player_data = df_filtered_85[df_filtered_85['PlayerID'] == player]
        player_max[player] = player_data[metrics].max()
        player_avg[player] = player_data[metrics].mean()

    # Add relative values to the original DataFrame
    for metric in metrics:

        df_original[f"{metric}_Rel"] = np.nan

        for player in df_original['PlayerID'].unique():
            if player in player_max and player in player_avg:
                max_value = player_max[player][metric]
                avg_value = player_avg[player][metric]

                # Compute relative value
                df_original.loc[df_original['PlayerID'] == player, f"{metric}_Rel"] = (
                    (df_original.loc[df_original['PlayerID'] == player, metric] * 100) /
                    ((max_value + avg_value) / 2)
                ).round(2)

    return df_original

Call function to calculate relative values

In [168]:
metrics_rel = ['>19.8', '>25']

# Avoid for now
df_rel = calculate_relative_values(df_filtered, metrics_rel)

# df_rel = df_filtered.copy()
df_rel.info()

<class 'pandas.core.frame.DataFrame'>
Index: 4720 entries, 3 to 7798
Data columns (total 21 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   DATE                4720 non-null   datetime64[ns]
 1   Session             4505 non-null   object        
 2   Injury              4720 non-null   float64       
 3   season              4720 non-null   object        
 4   preseason-season    4720 non-null   object        
 5   TD                  4720 non-null   float64       
 6   >19.8               4720 non-null   float64       
 7   >25                 4720 non-null   float64       
 8   ACC                 4720 non-null   float64       
 9   DEC                 4720 non-null   float64       
 10  PlayerID            4720 non-null   float64       
 11  Player Position     4720 non-null   object        
 12  Max Speed           4720 non-null   float64       
 13  Sprints             4720 non-null   float64       
 1

Merge with new dataset

In [169]:
# Rename columns in df_new
df_new["Date"] = pd.to_datetime(df_new["Date"], dayfirst=True)
df_new.rename(columns={'Date': 'DATE', '>19.8_Rel-1': '>19.8_Rel', '>25_Rel-1': '>25_Rel'}, inplace=True)
df_new = df_new[df_new["Session"].isin(['MD', 'MD-1', 'MD-2', 'MD-3', 'MD-5', 'MD+3', 'MD+2', 'MD-4', 'MD+1'])]

# Concatenate the dataframes
df_rel_concatenated = pd.concat([df_rel, df_new], axis=0)
df_rel_concatenated.info()

<class 'pandas.core.frame.DataFrame'>
Index: 7641 entries, 3 to 3250
Data columns (total 21 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   DATE                7641 non-null   datetime64[ns]
 1   Session             7426 non-null   object        
 2   Injury              7641 non-null   float64       
 3   season              4720 non-null   object        
 4   preseason-season    4720 non-null   object        
 5   TD                  7641 non-null   float64       
 6   >19.8               7641 non-null   float64       
 7   >25                 7641 non-null   float64       
 8   ACC                 7641 non-null   float64       
 9   DEC                 7641 non-null   float64       
 10  PlayerID            7641 non-null   float64       
 11  Player Position     4720 non-null   object        
 12  Max Speed           4720 non-null   float64       
 13  Sprints             7641 non-null   float64       
 1

In [170]:
md_values = ['MD', 'MD(HOME)', 'MD(AWAY)']
df_rel_concatenated['Session'] = df_rel_concatenated['Session'].replace(md_values, 'MD')

In [171]:
# Dictionary to map old session values to new ones
session_mapping = {
    'M-1': 'MD-1', 'M-2': 'MD-2', 'M-3': 'MD-3', 'M-5': 'MD-5',
    'M+3': 'MD+3', 'M+2': 'MD+2', 'M-4': 'MD-4', 'M+1': 'MD+1'
}

# Replace the specified values with the new 'MD' values
df_rel_concatenated['Session'] = df_rel_concatenated['Session'].replace(session_mapping)

# Verify the unique values in the 'Session' column
df_rel_concatenated['Session'].unique()

array(['MD', 'MD-1', 'MD-2', 'MD-3', 'MD-5', 'MD+3', 'MD+2', 'MD-4',
       'MD+1', nan], dtype=object)

# Function for densities

In [172]:
# cols_mins = ['TD', 'HSR', '+25 Km/h', 'ACC', 'DEC', 'Sprints']

# # Create new columns by dividing by the 'Minutes' column
# for col in cols_mins:
#     df_rel[f'{col}/Mins'] = df_rel[col] / df_rel['Mins']

In [173]:
# df_rel['Sprints/Mins'] = df_rel['Sprints/Mins'].fillna(0)

# Player loads

## Calculate loads

In [174]:
def calcular_acumulado(df, columnas_calcular, dias):
    # Columns to exclude when the day is 3
    excluded_columns_3_days = ['TD_Rel', '>19.8_Rel', '>25_Rel', 'ACC_Rel', 'DEC_Rel', '% Max Speed']

    # Create an empty list to store processed player DataFrames
    processed_players = []

    # Process each player separately
    for player_id in df['PlayerID'].unique():
        # Filter data for the current player
        player_data = df[df['PlayerID'] == player_id].copy()

        # Create a full date range for the player (from the first to the last recorded date)
        full_date_range = pd.date_range(start=player_data['DATE'].min(), end=player_data['DATE'].max(), freq='D')

        # Set 'DATE' as the index and reindex to fill missing dates with zeros
        player_data = player_data.set_index('DATE').reindex(full_date_range, fill_value=0).reset_index()
        player_data.rename(columns={'index': 'DATE'}, inplace=True)
        player_data['PlayerID'] = player_id

        # Perform rolling calculations for each metric, excluding the current day
        for dia in dias:
            for col in columnas_calcular:
                # Skip excluded columns when the day is 3
                if dia != 1 and col in excluded_columns_3_days:
                    continue

                # Check if the column exists to avoid errors
                if col in player_data.columns:
                    if dia in [7, 21]:
                        # Rolling sum, mean, and std for 7 and 21 days
                        player_data[f'{col}-{dia}'] = (
                            player_data[col].shift(1).rolling(window=dia, min_periods=1).sum()
                        )
                        player_data[f'{col}-{dia}-avg'] = (
                            player_data[col].shift(1).rolling(window=dia, min_periods=1).mean()
                        )
                        player_data[f'{col}-{dia}-std'] = (
                            player_data[col].shift(1).rolling(window=dia, min_periods=1).std()
                        )
                    else:
                        # Only rolling sum for other periods
                        player_data[f'{col}-{dia}'] = (
                            player_data[col].shift(1).rolling(window=dia, min_periods=1).sum()
                        )

        # Drop rows where all calculated values are zero (rest days)
        mask_non_zero = (player_data[columnas_calcular].sum(axis=1) > 0)
        player_data = player_data[mask_non_zero]

        # Append the processed player's data to the list
        processed_players.append(player_data)

    # Concatenate all processed player DataFrames into a single DataFrame
    df_resultado = pd.concat(processed_players, ignore_index=True)

    return df_resultado


In [216]:
cols_calculate = ['TD', '>19.8', '>25', 'ACC', 'DEC', 'Sprints', 'Mins', '>19.8_Rel', '>25_Rel', '% Max Speed']

cumulative_df = calcular_acumulado(df_rel_concatenated, cols_calculate, [1,3,7,21]) # loads -1, -3, -7, -21

In [217]:
cumulative_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7641 entries, 0 to 7640
Data columns (total 80 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   DATE                7641 non-null   datetime64[ns]
 1   Session             7426 non-null   object        
 2   Injury              7641 non-null   float64       
 3   season              4720 non-null   object        
 4   preseason-season    4720 non-null   object        
 5   TD                  7641 non-null   float64       
 6   >19.8               7641 non-null   float64       
 7   >25                 7641 non-null   float64       
 8   ACC                 7641 non-null   float64       
 9   DEC                 7641 non-null   float64       
 10  PlayerID            7641 non-null   float64       
 11  Player Position     4720 non-null   object        
 12  Max Speed           4720 non-null   float64       
 13  Sprints             7641 non-null   float64     

In [218]:
cumulative_df.sort_values(by="DATE", ascending=False).head()

,DATE,Session,Injury,season,preseason-season,TD,>19.8,>25,ACC,DEC,PlayerID,Player Position,Max Speed,Sprints,Mins,Max Speed Season,Avg Speed Season,% Max Speed,Speed Diff Max Avg,>19.8_Rel,>25_Rel,TD-1,>19.8-1,>25-1,ACC-1,DEC-1,Sprints-1,Mins-1,>19.8_Rel-1,>25_Rel-1,% Max Speed-1,TD-3,>19.8-3,>25-3,ACC-3,DEC-3,Sprints-3,Mins-3,TD-7,TD-7-avg,TD-7-std,>19.8-7,>19.8-7-avg,>19.8-7-std,>25-7,>25-7-avg,>25-7-std,ACC-7,ACC-7-avg,ACC-7-std,DEC-7,DEC-7-avg,DEC-7-std,Sprints-7,Sprints-7-avg,Sprints-7-std,Mins-7,Mins-7-avg,Mins-7-std,TD-21,TD-21-avg,TD-21-std,>19.8-21,>19.8-21-avg,>19.8-21-std,>25-21,>25-21-avg,>25-21-std,ACC-21,ACC-21-avg,ACC-21-std,DEC-21,DEC-21-avg,DEC-21-std,Sprints-21,Sprints-21-avg,Sprints-21-std,Mins-21,Mins-21-avg,Mins-21-std
7640,2025-02-01,MD,0.0,NaN,NaN,10383.19,327.09,133.93,121.0,135.0,177.0,NaN,NaN,9.0,116.54,NaN,NaN,94.92,NaN,0.00,0.00,2673.04,30.45,0.00,53.0,48.0,0.0,62.0,0.00,0.0,70.02,8484.97,359.24,166.74,113.0,113.0,8.0,158.0,17505.93,2500.847143,2057.319044,667.43,95.347143,122.508898,212.16,30.308571,62.125843,219.0,31.285714,30.641553,205.0,29.285714,27.505844,11.0,1.571429,3.047247,294.87,42.124286,31.656355,55719.54,3277.620000,2743.546004,2123.61,124.918235,140.945838,494.05,29.061765,45.230420,785.0,46.176471,37.954307,760.0,44.705882,39.504374,33.0,1.941176,2.585309,869.21,51.130000,35.546811
5695,2025-02-01,MD,0.0,NaN,NaN,10981.08,516.02,64.50,132.0,138.0,45.0,NaN,NaN,7.0,98.92,NaN,NaN,84.14,NaN,39.33,0.00,2076.10,6.08,0.00,32.0,25.0,0.0,31.0,0.00,0.0,63.16,8865.06,436.77,88.28,99.0,90.0,6.0,147.0,24785.39,3540.770000,3733.293339,962.68,137.525714,176.799624,128.46,18.351429,28.610946,278.0,39.714286,42.097958,291.0,41.571429,50.228999,10.0,1.428571,1.902379,323.17,46.167143,37.603576,78714.03,3748.287143,3463.152493,2859.37,136.160476,182.780364,599.58,28.551429,48.409010,907.0,43.190476,40.460622,899.0,42.809524,44.196854,37.0,1.761905,2.844376,1082.66,51.555238,36.532507
7615,2025-02-01,MD,0.0,NaN,NaN,3082.34,336.42,84.81,23.0,10.0,104.0,NaN,NaN,2.0,40.16,NaN,NaN,82.17,NaN,33.02,0.00,2086.96,2.94,0.00,43.0,39.0,0.0,31.0,0.00,0.0,61.48,6278.08,334.03,105.08,87.0,84.0,7.0,93.0,10941.40,1563.057143,2104.178735,417.15,59.592857,123.611627,105.08,15.011429,39.716507,148.0,21.142857,27.008817,117.0,16.714286,21.132237,7.0,1.000000,2.645751,161.00,23.000000,30.892286,36876.34,1756.016190,1892.431056,1932.20,92.009524,140.342162,377.11,17.957619,34.389001,493.0,23.476190,28.671622,379.0,18.047619,24.003075,22.0,1.047619,2.060975,582.00,27.714286,30.893596
4845,2025-02-01,MD,0.0,NaN,NaN,9682.92,678.90,216.61,106.0,103.0,136.0,NaN,NaN,13.0,98.92,NaN,NaN,87.80,NaN,55.15,24.47,2362.12,127.67,20.86,44.0,34.0,2.0,62.0,8.71,0.0,84.47,8611.60,494.34,334.16,101.0,93.0,14.0,178.0,19417.92,2773.988571,2325.525781,823.84,117.691429,96.499676,507.60,72.514286,108.216720,231.0,33.000000,30.697448,209.0,29.857143,26.124245,22.0,3.142857,3.891382,340.97,48.710000,34.850424,56489.87,2689.993810,1715.913934,2221.65,105.792857,125.109528,1060.78,50.513333,81.778599,757.0,36.047619,27.043439,638.0,30.380952,23.414688,55.0,2.619048,3.293572,1038.16,49.436190,27.920275
7582,2025-02-01,MD,0.0,NaN,NaN,1241.67,367.42,8.13,18.0,9.0,4.0,NaN,NaN,1.0,8.62,NaN,NaN,79.42,NaN,0.00,0.00,2660.85,17.99,0.00,53.0,47.0,0.0,62.0,0.00,0.0,66.61,6943.57,314.69,72.40,108.0,97.0,3.0,124.0,15807.07,2258.152857,2237.615587,838.35,119.764286,163.487468,134.48,19.211429,30.057604,238.0,34.000000,35.707142,202.0,28.857143,31.371051,6.0,0.857143,1.214986,259.96,37.137143,35.400730,72971.70,3474.842857,3850.598406,4040.53,192.406190,240.981871,794.03,37.810952,66.695186,870.0,41.428571,43.215242,720.0,34.285714,37.797014,45.0,2.142857,4.629100,1009.78,48.084762,40.378405


## For player loads, drop rows with NULL or 0 values

In [219]:
cumulative_df.columns


Index(['DATE', 'Session', 'Injury', 'season', 'preseason-season', 'TD',
       '>19.8', '>25', 'ACC', 'DEC', 'PlayerID', 'Player Position',
       'Max Speed', 'Sprints', 'Mins', 'Max Speed Season', 'Avg Speed Season',
       '% Max Speed', 'Speed Diff Max Avg', '>19.8_Rel', '>25_Rel', 'TD-1',
       '>19.8-1', '>25-1', 'ACC-1', 'DEC-1', 'Sprints-1', 'Mins-1',
       '>19.8_Rel-1', '>25_Rel-1', '% Max Speed-1', 'TD-3', '>19.8-3', '>25-3',
       'ACC-3', 'DEC-3', 'Sprints-3', 'Mins-3', 'TD-7', 'TD-7-avg', 'TD-7-std',
       '>19.8-7', '>19.8-7-avg', '>19.8-7-std', '>25-7', '>25-7-avg',
       '>25-7-std', 'ACC-7', 'ACC-7-avg', 'ACC-7-std', 'DEC-7', 'DEC-7-avg',
       'DEC-7-std', 'Sprints-7', 'Sprints-7-avg', 'Sprints-7-std', 'Mins-7',
       'Mins-7-avg', 'Mins-7-std', 'TD-21', 'TD-21-avg', 'TD-21-std',
       '>19.8-21', '>19.8-21-avg', '>19.8-21-std', '>25-21', '>25-21-avg',
       '>25-21-std', 'ACC-21', 'ACC-21-avg', 'ACC-21-std', 'DEC-21',
       'DEC-21-avg', 'DEC-21-std', 'Spr

In [220]:
cols_drop = ['Max Speed','Speed Diff Max Avg', 'Max Speed Season', 'Avg Speed Season','Player Position']
cumulative_df.drop(columns=cols_drop, inplace=True)

Drop rows that contain more than 2 NULL values

In [221]:
# Exclude 'Session', 'preseason-season', and 'season' columns for counting NULLs
columns_to_check = cumulative_df.drop(columns=['Session', 'preseason-season', 'season']).columns

# Drop rows where 2 or more null values exist (excluding above cols)
cumulative_df = cumulative_df[cumulative_df[columns_to_check].notna().sum(axis=1) > (len(columns_to_check) - 2)]

# Display the resulting DataFrame
cumulative_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 7362 entries, 2 to 7640
Data columns (total 75 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   DATE              7362 non-null   datetime64[ns]
 1   Session           7147 non-null   object        
 2   Injury            7362 non-null   float64       
 3   season            4677 non-null   object        
 4   preseason-season  4677 non-null   object        
 5   TD                7362 non-null   float64       
 6   >19.8             7362 non-null   float64       
 7   >25               7362 non-null   float64       
 8   ACC               7362 non-null   float64       
 9   DEC               7362 non-null   float64       
 10  PlayerID          7362 non-null   float64       
 11  Sprints           7362 non-null   float64       
 12  Mins              7362 non-null   float64       
 13  % Max Speed       7362 non-null   float64       
 14  >19.8_Rel         7362 non-nu

In [222]:
# Count of rows containing 0 for each column
zero_counts = (cumulative_df == 0).sum()

# Creating a DataFrame for the results
result_df = pd.DataFrame({
    "Column Name": zero_counts.index,
    "Number of Rows with 0": zero_counts.values
})

result_df

,Column Name,Number of Rows with 0
0,DATE,0
1,Session,0
2,Injury,7317
3,season,0
4,preseason-season,0
...,...,...
70,Sprints-21-avg,218
71,Sprints-21-std,219
72,Mins-21,64
73,Mins-21-avg,64


Remove rows where 3 or more rows contain 0 for the loads

In [223]:
cumulative_df.columns

Index(['DATE', 'Session', 'Injury', 'season', 'preseason-season', 'TD',
       '>19.8', '>25', 'ACC', 'DEC', 'PlayerID', 'Sprints', 'Mins',
       '% Max Speed', '>19.8_Rel', '>25_Rel', 'TD-1', '>19.8-1', '>25-1',
       'ACC-1', 'DEC-1', 'Sprints-1', 'Mins-1', '>19.8_Rel-1', '>25_Rel-1',
       '% Max Speed-1', 'TD-3', '>19.8-3', '>25-3', 'ACC-3', 'DEC-3',
       'Sprints-3', 'Mins-3', 'TD-7', 'TD-7-avg', 'TD-7-std', '>19.8-7',
       '>19.8-7-avg', '>19.8-7-std', '>25-7', '>25-7-avg', '>25-7-std',
       'ACC-7', 'ACC-7-avg', 'ACC-7-std', 'DEC-7', 'DEC-7-avg', 'DEC-7-std',
       'Sprints-7', 'Sprints-7-avg', 'Sprints-7-std', 'Mins-7', 'Mins-7-avg',
       'Mins-7-std', 'TD-21', 'TD-21-avg', 'TD-21-std', '>19.8-21',
       '>19.8-21-avg', '>19.8-21-std', '>25-21', '>25-21-avg', '>25-21-std',
       'ACC-21', 'ACC-21-avg', 'ACC-21-std', 'DEC-21', 'DEC-21-avg',
       'DEC-21-std', 'Sprints-21', 'Sprints-21-avg', 'Sprints-21-std',
       'Mins-21', 'Mins-21-avg', 'Mins-21-std'],
      

In [224]:
print("No Injury: " + str(cumulative_df[cumulative_df["Injury"] == 0].shape[0]))
print("Injury: " + str(cumulative_df[cumulative_df["Injury"] == 1].shape[0]))
print(f"%: {((cumulative_df[cumulative_df['Injury'] == 1].shape[0] / cumulative_df.shape[0]) * 100):.2f}%")

No Injury: 7317
Injury: 45
%: 0.61%


In [225]:
# Define columns to consider

columns_to_consider = ['TD-3', '>19.8-3', '>25-3',
       'ACC-3', 'DEC-3', 'Sprints-3', 'TD-7', '>19.8-7', '>25-7',
       'ACC-7', 'DEC-7', 'Sprints-7', 'TD-21', '>19.8-21', '>25-21',
       'ACC-21', 'DEC-21', 'Sprints-21']

# columns_to_consider = ['TD-7', '>19.8-7', '>25-7',
#        'ACC-7', 'DEC-7', 'Sprints-7', 'Mins-7', 'TD-21', '>19.8-21', '>25-21',
#        'ACC-21', 'DEC-21', 'Sprints-21', 'Mins-21']

# Count zero values in the specified columns for each row
zero_counts = (cumulative_df[columns_to_consider] == 0).sum(axis=1)

# Filter out rows with 3 or more zeros in the specified columns
cumulative_df = cumulative_df[zero_counts < 3]

In [226]:
# Count of rows containing 0 for each column
zero_counts = (cumulative_df == 0).sum()

# Creating a DataFrame for the results
result_df = pd.DataFrame({
    "Column Name": zero_counts.index,
    "Number of Rows with 0": zero_counts.values
})

result_df

,Column Name,Number of Rows with 0
0,DATE,0
1,Session,0
2,Injury,6555
3,season,0
4,preseason-season,0
...,...,...
70,Sprints-21-avg,0
71,Sprints-21-std,1
72,Mins-21,0
73,Mins-21-avg,0


In [227]:
print("No Injury: " + str(cumulative_df[cumulative_df["Injury"] == 0].shape[0]))
print("Injury: " + str(cumulative_df[cumulative_df["Injury"] == 1].shape[0]))
print(f"%: {((cumulative_df[cumulative_df['Injury'] == 1].shape[0] / cumulative_df.shape[0]) * 100):.2f}%")

No Injury: 6555
Injury: 42
%: 0.64%


## Calculate different load metrics

ACWR and MSWR values, trying to replicate the research paper

In [228]:
cumulative_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 6597 entries, 2 to 7640
Data columns (total 75 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   DATE              6597 non-null   datetime64[ns]
 1   Session           6417 non-null   object        
 2   Injury            6597 non-null   float64       
 3   season            4141 non-null   object        
 4   preseason-season  4141 non-null   object        
 5   TD                6597 non-null   float64       
 6   >19.8             6597 non-null   float64       
 7   >25               6597 non-null   float64       
 8   ACC               6597 non-null   float64       
 9   DEC               6597 non-null   float64       
 10  PlayerID          6597 non-null   float64       
 11  Sprints           6597 non-null   float64       
 12  Mins              6597 non-null   float64       
 13  % Max Speed       6597 non-null   float64       
 14  >19.8_Rel         6597 non-nu

In [229]:
min_threshold = 1e-5  # Small constant to replace zeros

cumulative_df['>19.8-7-std'] = cumulative_df['>19.8-7-std'].replace(0, min_threshold)
cumulative_df['>25-7-std'] = cumulative_df['>25-7-std'].replace(0, min_threshold)
cumulative_df['>19.8-21-avg'] = cumulative_df['>19.8-21-avg'].replace(0, min_threshold)
cumulative_df['>25-21-avg'] = cumulative_df['>25-21-avg'].replace(0, min_threshold)

In [230]:
def calculate_metrics_loads(df, metrics):

  # Calculate ACWR, MSWR for each metric
  for metric in metrics:

      # Calculate 7-day and 28-day averages for ACWR
      df[f'{metric}_ACWR'] = df[f'{metric}-7-avg'] / df[f'{metric}-21-avg']

      # Calculate mean and standard deviation for MSWR
      df[f'{metric}_MSWR'] = df[f'{metric}-7-avg'] / df[f'{metric}-7-std']

  return df

cols_calculate = ['TD', '>19.8', '>25', 'ACC', 'DEC']
cumulative_df = calculate_metrics_loads(cumulative_df, cols_calculate)

cumulative_df.drop(columns=['TD-7-avg', 'TD-7-std',
'>19.8-7-avg', '>19.8-7-std', '>25-7-avg',
       '>25-7-std', 'ACC-7-avg', 'ACC-7-std', 'DEC-7-avg',
       'DEC-7-std', 'Sprints-7-avg', 'Sprints-7-std',
       'Mins-7-avg', 'Mins-7-std', 'TD-21-avg', 'TD-21-std', '>19.8-21-avg', '>19.8-21-std', '>25-21-avg',
       '>25-21-std', 'ACC-21-avg', 'ACC-21-std',
       'DEC-21-avg', 'DEC-21-std', 'Sprints-21-avg',
       'Sprints-21-std', 'Mins-21-avg', 'Mins-21-std'] , inplace=True)

In [231]:
cumulative_df.describe()

/Users/tommylas/Documents/FootballAnalytics/WatfordFC/.venv/lib/python3.12/site-packages/pandas/core/nanops.py:1010: RuntimeWarning: invalid value encountered in subtract
  sqr = _ensure_numeric((avg - values) ** 2)
/Users/tommylas/Documents/FootballAnalytics/WatfordFC/.venv/lib/python3.12/site-packages/pandas/core/nanops.py:1010: RuntimeWarning: invalid value encountered in subtract
  sqr = _ensure_numeric((avg - values) ** 2)


,DATE,Injury,TD,>19.8,>25,ACC,DEC,PlayerID,Sprints,Mins,% Max Speed,>19.8_Rel,>25_Rel,TD-1,>19.8-1,>25-1,ACC-1,DEC-1,Sprints-1,Mins-1,>19.8_Rel-1,>25_Rel-1,% Max Speed-1,TD-3,>19.8-3,>25-3,ACC-3,DEC-3,Sprints-3,Mins-3,TD-7,>19.8-7,>25-7,ACC-7,DEC-7,Sprints-7,Mins-7,TD-21,>19.8-21,>25-21,ACC-21,DEC-21,Sprints-21,Mins-21,TD_ACWR,TD_MSWR,>19.8_ACWR,>19.8_MSWR,>25_ACWR,>25_MSWR,ACC_ACWR,ACC_MSWR,DEC_ACWR,DEC_MSWR
count,6597,6597.000000,6597.000000,6597.000000,6597.000000,6597.000000,6597.000000,6597.000000,6597.000000,6597.000000,6597.000000,6597.00000,6597.000000,6597.00000,6597.000000,6597.000000,6597.000000,6597.000000,6597.000000,6597.000000,6597.000000,6597.000000,6597.000000,6597.000000,6597.000000,6597.000000,6597.000000,6597.000000,6597.000000,6597.000000,6597.000000,6597.000000,6597.000000,6597.000000,6597.000000,6597.000000,6597.000000,6597.000000,6597.000000,6597.000000,6597.000000,6597.000000,6597.000000,6597.000000,6597.000000,6597.000000,6597.000000,6597.000000,6597.000000,6597.000000,6597.000000,6597.000000,6597.000000,6597.000000
mean,2023-06-02 15:53:00.900409344,0.006367,4670.070903,183.564684,42.073345,50.633927,42.734290,21800.491587,9.877862,67.371731,77.234056,28.09804,16.186095,2811.23728,97.860930,18.696803,32.820761,25.713901,5.043808,45.785307,14.873492,6.933252,53.752892,9520.999720,380.312329,83.306890,106.011547,87.258227,19.294311,139.934698,21693.733583,861.276027,190.833899,238.896569,198.690119,43.729596,315.664824,58837.281938,2341.217608,518.003030,642.226605,537.224650,117.528747,850.894743,1.140718,1.299831,1.144500,0.813609,1.174887,0.579606,1.153124,inf,1.152609,inf
min,2021-08-12 00:00:00,0.000000,0.300000,0.000000,0.000000,0.000000,0.000000,4.000000,0.000000,4.040000,0.000000,0.00000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,492.000000,1.000000,0.000000,2.000000,0.000000,0.000000,4.040000,1439.000000,2.000000,0.000000,10.000000,3.000000,0.000000,8.680000,2794.000000,10.000000,1.000000,15.000000,4.000000,1.000000,35.000000,0.164586,0.377964,0.002640,0.377964,0.000000,0.000000,0.081213,0.377964,0.044118,0.377964
25%,2022-05-15 00:00:00,0.000000,2870.700000,36.080000,0.000000,30.000000,22.000000,134.000000,1.000000,53.000000,69.866201,0.46000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,6838.000000,198.000000,17.000000,71.000000,57.000000,4.000000,99.520000,18222.870000,592.940000,86.000000,187.000000,151.000000,11.000000,270.580000,49009.000000,1682.000000,260.000000,491.000000,396.000000,33.000000,698.220000,0.900579,0.892298,0.834100,0.620713,0.740536,0.461318,0.891975,0.880173,0.885083,0.799208
50%,2023-01-18 00:00:00,0.000000,4097.940000,113.000000,9.510000,47.000000,38.000000,17316.000000,5.000000,67.970000,78.808674,14.41000,0.570000,2944.00000,38.000000,0.000000,31.000000,22.000000,1.000000,55.000000,1.330000,0.000000,70.140000,9634.000000,361.000000,63.000000,102.000000,85.000000,12.000000,141.000000,22259.000000,831.000000,164.040000,237.000000,198.000000,32.000000,325.650000,62834.000000,2362.000000,449.060000,660.000000,552.200000,77.000000,927.370000,1.025000,1.049657,1.021393,0.724030,1.047270,0.572469,1.025608,1.048348,1.034929,0.976805
75%,2024-10-01 00:00:00,0.000000,5697.000000,289.000000,61.000000,67.000000,57.000000,37709.000000,14.000000,82.910000,87.596236,44.00000,20.680000,4471.00000,136.000000,15.260000,53.000000,42.000000,7.000000,72.230000,19.570000,3.270000,80.700000,11903.220000,530.000000,126.000000,135.000000,113.400000,32.000000,171.960000,25724.000000,1109.000000,269.000000,288.000000,244.000000,71.000000,374.070000,71557.000000,2998.000000,733.980000,801.000000,678.000000,195.000000,1043.470000,1.213042,1.309556,1.298711,0.853229,1.439655,0.657279,1.249147,1.257395,1.263017,1.176239
max,2025-02-01 00:00:00,1.000000,13734.740000,1172.000000,438.460000,186.000000,173.000000,89871.000000,72.000000,141.810000,463.450000,327.00000,276.450000,13416.00000,1057

In [232]:
cumulative_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 6597 entries, 2 to 7640
Data columns (total 57 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   DATE              6597 non-null   datetime64[ns]
 1   Session           6417 non-null   object        
 2   Injury            6597 non-null   float64       
 3   season            4141 non-null   object        
 4   preseason-season  4141 non-null   object        
 5   TD                6597 non-null   float64       
 6   >19.8             6597 non-null   float64       
 7   >25               6597 non-null   float64       
 8   ACC               6597 non-null   float64       
 9   DEC               6597 non-null   float64       
 10  PlayerID          6597 non-null   float64       
 11  Sprints           6597 non-null   float64       
 12  Mins              6597 non-null   float64       
 13  % Max Speed       6597 non-null   float64       
 14  >19.8_Rel         6597 non-nu

# Do the shift for X previous days injury

If a player got injured X day, it means that the player has been loaded and could have happened the previous session or maybe in the previous 2 sessions.
To add more Injury columns, make the previous 2 days as Injury = 1.

In [233]:
cumulative_df

,DATE,Session,Injury,season,preseason-season,TD,>19.8,>25,ACC,DEC,PlayerID,Sprints,Mins,% Max Speed,>19.8_Rel,>25_Rel,TD-1,>19.8-1,>25-1,ACC-1,DEC-1,Sprints-1,Mins-1,>19.8_Rel-1,>25_Rel-1,% Max Speed-1,TD-3,>19.8-3,>25-3,ACC-3,DEC-3,Sprints-3,Mins-3,TD-7,>19.8-7,>25-7,ACC-7,DEC-7,Sprints-7,Mins-7,TD-21,>19.8-21,>25-21,ACC-21,DEC-21,Sprints-21,Mins-21,TD_ACWR,TD_MSWR,>19.8_ACWR,>19.8_MSWR,>25_ACWR,>25_MSWR,ACC_ACWR,ACC_MSWR,DEC_ACWR,DEC_MSWR
2,2022-10-31,MD+2,0.0,2022-2023,SEASON,4438.00,46.00,0.00,81.0,46.0,57090.0,7.0,59.90,73.833432,12.53,0.00,0.00,0.00,0.00,0.0,0.0,0.0,0.00,0.00,0.00,0.000000,2417.00,12.00,1.00,18.0,18.0,1.0,53.45,6121.00,22.00,1.00,61.0,60.0,2.0,117.36,6121.00,22.00,1.00,61.0,60.0,2.0,117.36,1.000000,0.830104,1.000000,0.858956,1.000000,0.500000,1.000000,0.749270,1.000000,0.753778
3,2022-11-01,MD-1,0.0,2022-2023,SEASON,4023.00,151.00,0.00,29.0,17.0,57090.0,16.0,66.75,69.994093,41.12,0.00,4438.00,46.00,0.00,81.0,46.0,7.0,59.90,12.53,0.00,73.833432,4438.00,46.00,0.00,81.0,46.0,7.0,59.90,10559.00,68.00,1.00,142.0,106.0,9.0,177.26,10559.00,68.00,1.00,142.0,106.0,9.0,177.26,1.000000,1.025615,1.000000,0.717981,1.000000,0.447214,1.000000,0.828407,1.000000,0.958501
4,2022-11-02,MD,0.0,2022-2023,SEASON,3093.00,124.00,4.00,37.0,22.0,57090.0,14.0,31.42,77.554637,33.77,3.44,4023.00,151.00,0.00,29.0,17.0,16.0,66.75,41.12,0.00,69.994093,8461.00,197.00,0.00,110.0,63.0,23.0,126.65,14582.00,219.00,1.00,171.0,123.0,25.0,244.01,14582.00,219.00,1.00,171.0,123.0,25.0,244.01,1.000000,1.215083,1.000000,0.622909,1.000000,0.408248,1.000000,0.929419,1.000000,1.032382
5,2022-11-03,MD+1,0.0,2022-2023,SEASON,3908.00,27.00,0.00,47.0,39.0,57090.0,4.0,61.80,69.049025,7.35,0.00,3093.00,124.00,4.00,37.0,22.0,14.0,31.42,33.77,3.44,77.554637,11554.00,321.00,4.00,147.0,85.0,37.0,158.07,17675.00,343.00,5.00,208.0,145.0,39.0,275.43,17675.00,343.00,5.00,208.0,145.0,39.0,275.43,1.000000,1.370074,1.000000,0.779153,1.000000,0.477455,1.000000,1.054584,1.000000,1.142181
6,2022-11-04,MD-1,0.0,2022-2023,SEASON,4014.00,138.00,0.00,13.0,13.0,57090.0,7.0,70.48,70.998228,37.58,0.00,3908.00,27.00,0.00,47.0,39.0,4.0,61.80,7.35,0.00,69.049025,11024.00,302.00,4.00,113.0,78.0,34.0,159.97,17879.00,360.00,5.00,212.0,142.0,42.0,273.32,21583.00,370.00,5.00,255.0,184.0,43.0,337.23,0.946724,1.368645,1.111969,0.837008,1.142857,0.477455,0.950140,1.061576,0.881988,1.154192
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7636,2025-01-28,MD-4,0.0,NaN,NaN,5175.50,131.56,3.29,73.0,67.0,177.0,0.0,79.00,80.440000,0.00,0.00,0.00,0.00,0.00,0.0,0.0,0.0,0.00,0.00,0.00,0.000000,3845.46,176.63,42.13,33.0,25.0,3.0,57.87,17578.75,723.11,123.46,228.0,232.0,9.0,251.89,42059.07,1632.81,324.02,599.0,580.0,25.0,632.21,0.776200,0.852903,0.822459,0.792901,0.707619,0.798871,0.706892,0.816170,0.742857,0.748339
7637,2025-01-29,MD-3,0.0,NaN,NaN,4241.49,322.83,166.74,57.0,55.0,177.0,8.0,62.00,102.630000,0.00,0.00,5175.50,131.56,3.29,73.0,67.0,0.0,79.00,0.00,0.00,80.440000,5175.50,131.56,3.29,73.0,67.0,0.0,79.00,14789.68,514.66,82.94,191.0,175.0,5.0,217.87,47234.57,1764.37,327.31,672.0,647.0,25.0,711.21,0.626223,0.973470,0.583392,0.895977,0.506798,0.617260,0.568452,0.945259,0.540958,0.945361
7638,2025-01-30,MD-2,0.0,NaN,NaN,1570.44,5.96,0.00,3.0,10.0,177.0,0.0,34.00,62.750000,0.00,0.00,4241.49,322.83,166.74,57.0,55.0,8.0,62.00,0.00,0.00,102.630000,9416.99,454.39,170.03,130.0,122.0,8.0,141.00,19031.17,837.49,249.68,248.0,230.0,13.0,279.87,51476.06,2087.20,494.05,729.0,702.0,33.0,773.21,0.792234,1.312088,0.859822,1.021781,1.082944,0.587846,0.728983,1.269399,0.702076,1.266415
7639,2025-01-31,MD-1,0.0,NaN,NaN,2673.04,30.45,0.00,53.0,48.0,177.0,0.0,62.00,70.020000,0.00,0.00,1570.44,5.96,0.00,3.0,10.0,0.0,34.00,0.00,0.00,62.750000,10987.43,460.35,170.03,133.0,132.0,8.0,175.00,16911.34,675.33,212.16,199.0,196.0,11.0,268.87,

In [234]:
def shift_injuries_x_days(df, x):
    copy_df = df.copy()

    # Iterate over the df to shift injury labels
    for idx, row in copy_df.iterrows():
        if row['Injury'] == 1:
            # Get the player's ID and the injury date
            player_id = row['PlayerID']
            injury_date = row['DATE']

            # Shift injury labels for the current day and the previous x days
            for i in range(0, x + 1):
                prev_date = pd.to_datetime(injury_date) - pd.Timedelta(days=i)
                mask = (copy_df['PlayerID'] == player_id) & (copy_df['DATE'] == prev_date)
                copy_df.loc[mask, 'Injury'] = 1

    return copy_df

In [235]:
# One previous day
# df_injury_shifted = shift_injuries_x_days(cumulative_df, 1)
df_injury_shifted = cumulative_df.copy()

In [236]:
print("No Injury: " + str(df_injury_shifted[df_injury_shifted["Injury"] == 0].shape[0]))
print("Injury: " + str(df_injury_shifted[df_injury_shifted["Injury"] == 1].shape[0]))
print(f"%: {((df_injury_shifted[df_injury_shifted['Injury'] == 1].shape[0] / df_injury_shifted.shape[0]) * 100):.2f}%")

No Injury: 6555
Injury: 42
%: 0.64%


In [237]:
df_injury_shifted.head()

,DATE,Session,Injury,season,preseason-season,TD,>19.8,>25,ACC,DEC,PlayerID,Sprints,Mins,% Max Speed,>19.8_Rel,>25_Rel,TD-1,>19.8-1,>25-1,ACC-1,DEC-1,Sprints-1,Mins-1,>19.8_Rel-1,>25_Rel-1,% Max Speed-1,TD-3,>19.8-3,>25-3,ACC-3,DEC-3,Sprints-3,Mins-3,TD-7,>19.8-7,>25-7,ACC-7,DEC-7,Sprints-7,Mins-7,TD-21,>19.8-21,>25-21,ACC-21,DEC-21,Sprints-21,Mins-21,TD_ACWR,TD_MSWR,>19.8_ACWR,>19.8_MSWR,>25_ACWR,>25_MSWR,ACC_ACWR,ACC_MSWR,DEC_ACWR,DEC_MSWR
2,2022-10-31,MD+2,0.0,2022-2023,SEASON,4438.0,46.0,0.0,81.0,46.0,57090.0,7.0,59.90,73.833432,12.53,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.00,0.00,0.000000,2417.0,12.0,1.0,18.0,18.0,1.0,53.45,6121.0,22.0,1.0,61.0,60.0,2.0,117.36,6121.0,22.0,1.0,61.0,60.0,2.0,117.36,1.000000,0.830104,1.000000,0.858956,1.000000,0.500000,1.00000,0.749270,1.000000,0.753778
3,2022-11-01,MD-1,0.0,2022-2023,SEASON,4023.0,151.0,0.0,29.0,17.0,57090.0,16.0,66.75,69.994093,41.12,0.00,4438.0,46.0,0.0,81.0,46.0,7.0,59.90,12.53,0.00,73.833432,4438.0,46.0,0.0,81.0,46.0,7.0,59.90,10559.0,68.0,1.0,142.0,106.0,9.0,177.26,10559.0,68.0,1.0,142.0,106.0,9.0,177.26,1.000000,1.025615,1.000000,0.717981,1.000000,0.447214,1.00000,0.828407,1.000000,0.958501
4,2022-11-02,MD,0.0,2022-2023,SEASON,3093.0,124.0,4.0,37.0,22.0,57090.0,14.0,31.42,77.554637,33.77,3.44,4023.0,151.0,0.0,29.0,17.0,16.0,66.75,41.12,0.00,69.994093,8461.0,197.0,0.0,110.0,63.0,23.0,126.65,14582.0,219.0,1.0,171.0,123.0,25.0,244.01,14582.0,219.0,1.0,171.0,123.0,25.0,244.01,1.000000,1.215083,1.000000,0.622909,1.000000,0.408248,1.00000,0.929419,1.000000,1.032382
5,2022-11-03,MD+1,0.0,2022-2023,SEASON,3908.0,27.0,0.0,47.0,39.0,57090.0,4.0,61.80,69.049025,7.35,0.00,3093.0,124.0,4.0,37.0,22.0,14.0,31.42,33.77,3.44,77.554637,11554.0,321.0,4.0,147.0,85.0,37.0,158.07,17675.0,343.0,5.0,208.0,145.0,39.0,275.43,17675.0,343.0,5.0,208.0,145.0,39.0,275.43,1.000000,1.370074,1.000000,0.779153,1.000000,0.477455,1.00000,1.054584,1.000000,1.142181
6,2022-11-04,MD-1,0.0,2022-2023,SEASON,4014.0,138.0,0.0,13.0,13.0,57090.0,7.0,70.48,70.998228,37.58,0.00,3908.0,27.0,0.0,47.0,39.0,4.0,61.80,7.35,0.00,69.049025,11024.0,302.0,4.0,113.0,78.0,34.0,159.97,17879.0,360.0,5.0,212.0,142.0,42.0,273.32,21583.0,370.0,5.0,255.0,184.0,43.0,337.23,0.946724,1.368645,1.111969,0.837008,1.142857,0.477455,0.95014,1.061576,0.881988,1.154192


# One Hot Encoding

One Hot Encoding for Match Day

In [238]:
# Step 2: Sort by PlayerID and Date
df_injury_shifted.sort_values(by=['PlayerID', 'DATE'], inplace=True)

# Step 3: Shift the Session column by -1 for each PlayerID
df_injury_shifted['Session'] = df_injury_shifted.groupby('PlayerID')['Session'].shift(1)

# Step 2: Perform one-hot encoding
final_encoded_final_df = pd.get_dummies(df_injury_shifted, columns=['Session'], prefix='Session')

one_hot_columns = [col for col in final_encoded_final_df.columns if col.startswith('Session_')]
final_encoded_final_df[one_hot_columns] = final_encoded_final_df[one_hot_columns].astype(int)

final_encoded_final_df = final_encoded_final_df.dropna(subset=one_hot_columns)
# Final encoded DataFrame
print(final_encoded_final_df)

           DATE  Injury     season preseason-season        TD   >19.8     >25  \
7547 2024-11-28     0.0        NaN              NaN   5673.60  227.63  106.23   
7548 2024-11-29     0.0        NaN              NaN   3187.82   29.50    0.00   
7549 2024-11-30     0.0        NaN              NaN  11860.70  525.95  239.76   
7550 2024-12-02     0.0        NaN              NaN   3881.53  150.27    0.00   
7551 2024-12-03     0.0        NaN              NaN   3256.74   18.06    0.00   
...         ...     ...        ...              ...       ...     ...     ...   
3496 2023-04-21     0.0  2022-2023           SEASON   4192.00  155.00    0.00   
3497 2023-04-22     0.0  2022-2023           SEASON   4572.00  214.00  101.00   
3498 2023-04-24     0.0  2022-2023           SEASON   3788.00  136.00   12.00   
3499 2023-04-25     0.0  2022-2023           SEASON   4871.00   83.00    0.00   
3501 2023-05-06     0.0  2022-2023           SEASON   2981.00   66.00    0.00   

        ACC    DEC  PlayerI

In [239]:
final_encoded_final_df.head()

,DATE,Injury,season,preseason-season,TD,>19.8,>25,ACC,DEC,PlayerID,Sprints,Mins,% Max Speed,>19.8_Rel,>25_Rel,TD-1,>19.8-1,>25-1,ACC-1,DEC-1,Sprints-1,Mins-1,>19.8_Rel-1,>25_Rel-1,% Max Speed-1,TD-3,>19.8-3,>25-3,ACC-3,DEC-3,Sprints-3,Mins-3,TD-7,>19.8-7,>25-7,ACC-7,DEC-7,Sprints-7,Mins-7,TD-21,>19.8-21,>25-21,ACC-21,DEC-21,Sprints-21,Mins-21,TD_ACWR,TD_MSWR,>19.8_ACWR,>19.8_MSWR,>25_ACWR,>25_MSWR,ACC_ACWR,ACC_MSWR,DEC_ACWR,DEC_MSWR,Session_MD,Session_MD+1,Session_MD+2,Session_MD+3,Session_MD-1,Session_MD-2,Session_MD-3,Session_MD-4,Session_MD-5
7547,2024-11-28,0.0,NaN,NaN,5673.60,227.63,106.23,48.0,45.0,4.0,6.0,84.72,100.85,40.60,15.35,0.00,0.00,0.00,0.0,0.0,0.0,0.00,0.0,0.00,0.00,9643.96,568.92,70.39,116.0,115.0,8.0,98.66,12066.12,616.88,70.39,146.0,142.0,8.0,153.66,16919.31,832.51,103.42,208.0,193.0,11.0,207.81,2.139470,0.477850,2.222964,0.414198,2.041868,0.377964,2.105769,0.480378,2.207254,0.472199,0,0,0,0,0,0,0,0,0
7548,2024-11-29,0.0,NaN,NaN,3187.82,29.50,0.00,44.0,38.0,4.0,0.0,56.67,72.32,0.00,0.00,5673.60,227.63,106.23,48.0,45.0,6.0,84.72,40.6,15.35,100.85,5673.60,227.63,106.23,48.0,45.0,6.0,84.72,15317.56,796.55,176.62,164.0,160.0,14.0,183.38,22592.91,1060.14,209.65,256.0,238.0,17.0,292.53,2.033943,0.559804,2.254089,0.522261,2.527355,0.569359,1.921875,0.525684,2.016807,0.519999,0,0,0,0,0,1,0,0,0
7549,2024-11-30,0.0,NaN,NaN,11860.70,525.95,239.76,125.0,106.0,4.0,16.0,128.30,97.02,90.74,43.74,3187.82,29.50,0.00,44.0,38.0,0.0,56.67,0.0,0.00,72.32,8861.42,257.13,106.23,92.0,83.0,6.0,141.39,18505.38,826.05,176.62,208.0,198.0,14.0,240.05,25780.73,1089.64,209.65,300.0,276.0,17.0,349.20,2.153397,0.696509,2.274283,0.547367,2.527355,0.569359,2.080000,0.678267,2.152174,0.657824,0,0,0,0,1,0,0,0,0
7550,2024-12-02,0.0,NaN,NaN,3881.53,150.27,0.00,54.0,34.0,4.0,0.0,52.42,76.35,0.00,0.00,0.00,0.00,0.00,0.0,0.0,0.0,0.00,0.0,0.00,0.00,15048.52,555.45,239.76,169.0,144.0,16.0,184.97,30366.08,1352.00,416.38,333.0,304.0,30.0,368.35,37641.43,1615.59,449.41,425.0,382.0,33.0,477.50,2.420159,0.884554,2.510538,0.755778,2.779511,0.658507,2.350588,0.881363,2.387435,0.876564,1,0,0,0,0,0,0,0,0
7551,2024-12-03,0.0,NaN,NaN,3256.74,18.06,0.00,20.0,14.0,4.0,0.0,74.30,69.70,0.00,0.00,3881.53,150.27,0.00,54.0,34.0,0.0,52.42,0.0,0.00,76.35,15742.23,676.22,239.76,179.0,140.0,16.0,180.72,24603.65,933.35,345.99,271.0,223.0,22.0,322.11,41522.96,1765.86,449.41,479.0,416.0,33.0,529.92,1.777594,0.814913,1.585658,0.684837,2.309628,0.532634,1.697286,0.855423,1.608173,0.833889,0,0,0,0,0,1,0,0,0


# Export data

In [64]:
#encoded_final_df.to_excel('/content/drive/MyDrive/WATFORD FC/Datos GPS/datos_finales_tommy2.xlsx', index=False)

In [240]:
final_encoded_final_df.to_excel('data/new_data_no_injured.xlsx', index=False)

In [129]:
final_encoded_final_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 7641 entries, 7619 to 3501
Data columns (total 70 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   DATE                7641 non-null   datetime64[ns]
 1   Injury              7641 non-null   float64       
 2   season              4720 non-null   object        
 3   preseason-season    4720 non-null   object        
 4   TD                  7641 non-null   float64       
 5   >19.8               7641 non-null   float64       
 6   >25                 7641 non-null   float64       
 7   ACC                 7641 non-null   float64       
 8   DEC                 7641 non-null   float64       
 9   PlayerID            7641 non-null   float64       
 10  Player Position     4720 non-null   object        
 11  Max Speed           4720 non-null   float64       
 12  Sprints             7641 non-null   float64       
 13  Mins                7641 non-null   float64       